In [1]:
import pandas as pd
import numpy as np
from keras.models import Model, load_model
from keras.layers import LSTM, Dense, Dropout, Input, Conv1D, Flatten, concatenate
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import os
import joblib
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

# Definir look_back
look_back = 6  

# Cargar el archivo y eliminar filas donde el clúster es NaN
file_path = 'data/data.csv'   # Cambia esta ruta al archivo que has subido
data = pd.read_csv(file_path)
data = data.dropna(subset=['Cluster_Label'])

# Convertir 'year' y 'month_no' a una sola columna de tipo fecha
data['date'] = pd.to_datetime(data['year'].astype(str) + '-' + data['month_no'].astype(str))

# Seleccionar las columnas necesarias para la serie temporal
columns_to_keep = ['date', 'species', 'Cluster_Label', 'landed_w_kg',
                   'mean_temp_30m','mean_temp_10m','thetao_sfc=6',
                   'thetao_sfc=7.92956018447876','thetao_sfc=9.572997093200684','thetao_sfc=11.40499973297119',
                   'thetao_sfc=13.46714019775391','thetao_sfc=15.8100700378418','thetao_sfc=18.49555969238281',
                   'thetao_sfc=21.59881973266602','thetao_sfc=25.21141052246094','thetao_sfc=29.44473075866699']
data = data[columns_to_keep]


2024-07-30 20:27:24.176725: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-30 20:27:24.180230: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-30 20:27:24.220547: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 20:27:24.220574: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 20:27:24.221847: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
# Función para preparar los datos
def prepare_data(group, look_back=6):
    features = ['landed_w_kg','Cluster_Label', 'mean_temp_30m','mean_temp_10m','thetao_sfc=6',
                   'thetao_sfc=7.92956018447876','thetao_sfc=9.572997093200684','thetao_sfc=11.40499973297119',
                   'thetao_sfc=13.46714019775391','thetao_sfc=15.8100700378418','thetao_sfc=18.49555969238281',
                   'thetao_sfc=21.59881973266602','thetao_sfc=25.21141052246094','thetao_sfc=29.44473075866699']
    scaler = MinMaxScaler(feature_range=(0, 1))
    group_scaled = scaler.fit_transform(group[features])
    
    X, y = [], []
    for i in range(len(group_scaled) - look_back):
        a = group_scaled[i:(i + look_back)]
        X.append(a)
        y.append(group_scaled[i + look_back, 0])  # La primera columna es 'landed_w_kg'
    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))
    return X, y, scaler

# Crear el modelo LSTM
def create_lstm_model(input_shape):
    lstm_input = Input(shape=input_shape)
    x = LSTM(50, return_sequences=True)(lstm_input)
    x = Dropout(0.2)(x)
    x = LSTM(25, activation='relu', return_sequences=True)(x)
    x = Dropout(0.2)(x)
    x = LSTM(12, activation='linear')(x)
    lstm_output = Dense(1, activation='linear')(x)
    return Model(inputs=lstm_input, outputs=lstm_output)

# Crear el modelo DNN
def create_dnn_model(input_shape):
    dnn_input = Input(shape=(input_shape[0], input_shape[1]))
    x = Flatten()(dnn_input)
    x = Dense(10, activation='relu')(x)
    x = Dense(5, activation='relu')(x)
    dnn_output = Dense(1, activation='linear')(x)
    return Model(inputs=dnn_input, outputs=dnn_output)

# Crear el modelo CNN
def create_cnn_model(input_shape):
    cnn_input = Input(shape=input_shape)
    x = Conv1D(32, kernel_size=3, activation='relu')(cnn_input)
    x = Flatten()(x)
    cnn_output = Dense(1, activation='linear')(x)
    return Model(inputs=cnn_input, outputs=cnn_output)

# Crear el modelo Mixture of Experts (MoE)
def create_moe_model(input_shape):
    # Definir los expertos
    lstm_model = create_lstm_model(input_shape)
    dnn_model = create_dnn_model(input_shape)
    cnn_model = create_cnn_model(input_shape)
    
    # Definir el gating network
    moe_input = Input(shape=input_shape)
    x = Flatten()(moe_input)
    x = Dense(10, activation='relu')(x)
    gate_output = Dense(3, activation='softmax')(x)  # Tres expertos

    # Obtener las salidas de los expertos
    lstm_output = lstm_model(moe_input)
    dnn_output = dnn_model(moe_input)
    cnn_output = cnn_model(moe_input)
    
    # Combinar las salidas usando el gating network
    output = concatenate([lstm_output * gate_output[:, 0:1],
                          dnn_output * gate_output[:, 1:2],
                          cnn_output * gate_output[:, 2:3]], axis=1)
    output = Dense(1, activation='linear')(output)
    
    model = Model(inputs=moe_input, outputs=output)
    model.compile(optimizer=Adam(), loss='mean_squared_error')
    return model


In [3]:
# Función para entrenar y guardar el modelo MoE
def train_and_save_moe_model(data, species_name, cluster_label, look_back=look_back, epochs=50, batch_size=1):
    X, y, scaler = prepare_data(data, look_back)
    model = create_moe_model((look_back, X.shape[2]))
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)
    history = model.fit(X, y, validation_split=0.2, epochs=epochs, batch_size=batch_size, verbose=1, callbacks=[early_stopping])
    
    # Guardar el modelo y el scaler
    model_directory = 'modelos_moe'
    os.makedirs(model_directory, exist_ok=True)
    model_path = os.path.join(model_directory, f'{species_name}_cluster_{cluster_label}_moe_model.h5')
    scaler_path = os.path.join(model_directory, f'{species_name}_cluster_{cluster_label}_moe_scaler.pkl')
    model.save(model_path)
    joblib.dump(scaler, scaler_path)
    
    # Guardar los valores de monitoreo del entrenamiento
    train_moe_directory = 'train_moe'
    os.makedirs(train_moe_directory, exist_ok=True)
    history_path = os.path.join(train_moe_directory, f'{species_name}_cluster_{cluster_label}_training_history.csv')
    pd.DataFrame(history.history).to_csv(history_path, index=False)
    
    print(f'Modelo MoE, scaler y datos de entrenamiento guardados para {species_name} en clúster {cluster_label}')
    return model, scaler

# Función para hacer predicciones con bandas de confianza usando bootstrap
def predict_with_confidence_intervals(model, scaler, data, look_back=6, n_bootstrap=100, alpha=0.01):
    features = ['landed_w_kg','Cluster_Label','mean_temp_30m','mean_temp_10m','thetao_sfc=6',
                   'thetao_sfc=7.92956018447876','thetao_sfc=9.572997093200684','thetao_sfc=11.40499973297119',
                   'thetao_sfc=13.46714019775391','thetao_sfc=15.8100700378418','thetao_sfc=18.49555969238281',
                   'thetao_sfc=21.59881973266602','thetao_sfc=25.21141052246094','thetao_sfc=29.44473075866699']
    group_scaled = scaler.transform(data[features])

    X = []
    for i in range(len(group_scaled) - look_back):
        a = group_scaled[i:(i + look_back)]
        X.append(a)
    X = np.array(X)
    X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))
    
    predictions = model.predict(X)
    predictions = scaler.inverse_transform(np.hstack((predictions, X[:, -1, 1:])))[:, 0]
    
    # Bootstrap para bandas de confianza
    bootstrap_predictions = []
    for _ in range(n_bootstrap):
        indices = np.random.choice(range(len(X)), len(X), replace=True)
        X_sample = X[indices]
        pred_sample = model.predict(X_sample)
        pred_sample = scaler.inverse_transform(np.hstack((pred_sample, X_sample[:, -1, 1:])))[:, 0]
        bootstrap_predictions.append(pred_sample)
    
    bootstrap_predictions = np.array(bootstrap_predictions)
    lower_bound = np.percentile(bootstrap_predictions, 100 * alpha / 2, axis=0)
    upper_bound = np.percentile(bootstrap_predictions, 100 * (1 - alpha / 2), axis=0)
    
    return predictions, lower_bound, upper_bound



In [5]:
# Obtener listas de especies y clústeres únicos
unique_species = data['species'].unique()
unique_clusters = data['Cluster_Label'].unique()

# Definir la especie y clúster desde donde retomar el proceso y detenerlo
start_species = 'BANDERA'
start_cluster = 7.0
stop_species = 'BANDERA'
stop_cluster = 8.0

# Inicializar banderas
start_training = False
stop_training = False



for species_name in unique_species:
    for cluster_label in unique_clusters:
        # Activar la bandera cuando se alcance la especie y clúster deseados para iniciar
        if species_name == start_species and cluster_label == start_cluster:
            start_training = True
        
        # Detener el proceso cuando se alcance la especie y clúster deseados para detener
        if species_name == stop_species and cluster_label == stop_cluster:
            stop_training = True
        
        # Continuar solo si la bandera de inicio está activada y la de detener no lo está
        if start_training and not stop_training:
            # Filtrar los datos para obtener una especie y un clúster específico
            filtered_data = data[(data['species'] == species_name) & (data['Cluster_Label'] == cluster_label)]
            
            if len(filtered_data) >= look_back:
                filtered_data = filtered_data.sort_values('date')
                try:
                    print(f'Training model for species: {species_name}, cluster: {cluster_label}')
                    model, scaler = train_and_save_moe_model(filtered_data, species_name, cluster_label, look_back)

                    # Guardar los datos para las bandas de confianza
                    test_data_2023 = filtered_data[(filtered_data['date'].dt.year == 2023) & (filtered_data['date'].dt.month >= 1)]
                    if len(test_data_2023) >= look_back:
                        predictions, lower_bound, upper_bound = predict_with_confidence_intervals(model, scaler, test_data_2023, look_back)
                        real_values = test_data_2023['landed_w_kg'].values[look_back:]
                        
                        # Crear un DataFrame para los resultados
                        results_df = pd.DataFrame({
                            'date': test_data_2023['date'].values[look_back:],
                            'real_values': real_values,
                            'predictions': predictions,
                            'lower_bound': lower_bound,
                            'upper_bound': upper_bound
                        })
                        
                        # Guardar los resultados
                        results_directory = 'resultados_moe'
                        os.makedirs(results_directory, exist_ok=True)
                        results_path = os.path.join(results_directory, f'{species_name}_cluster_{cluster_label}_predictions.csv')
                        results_df.to_csv(results_path, index=False)
                        
                        # Agrupar por mes y sumar los valores
                        monthly_totals = results_df.set_index('date').resample('M').sum()
                        
                        # Calcular MSE y R2 para los totales mensuales
                        mse = mean_squared_error(monthly_totals['real_values'], monthly_totals['predictions'])
                        r2 = r2_score(monthly_totals['real_values'], monthly_totals['predictions'])
                        print(f'MSE Mensual para {species_name} en clúster {cluster_label}: {mse}, R2 Mensual: {r2}')
                        
                        # Crear figura
                        plt.figure(figsize=(10, 6))
                        plt.plot(monthly_totals.index, monthly_totals['real_values'], color='blue', label='Real')
                        plt.plot(monthly_totals.index, monthly_totals['predictions'], color='red', linestyle='--', label='Predicho')
                        plt.fill_between(monthly_totals.index, monthly_totals['lower_bound'], monthly_totals['upper_bound'], color='gray', alpha=0.2, label='Intervalo de Confianza 95%')
                        plt.title(f'Totales Mensuales de {species_name} en clúster {cluster_label} para los Últimos Seis Meses de 2023')
                        plt.xlabel('Fecha')
                        plt.ylabel('Peso Desembarcado (kg)')
                        plt.legend()
                        plot_path = os.path.join(results_directory, f'{species_name}_cluster_{cluster_label}_plot.png')
                        plt.savefig(plot_path)
                        plt.close()
                    else:
                        print(f'No se tienen datos suficientes para {species_name} en clúster {cluster_label} en los últimos seis meses de 2023')
                except Exception as e:
                    print(f'Error entrenando el modelo para especie: {species_name}, clúster: {cluster_label}. Error: {e}')
        
        # Salir del bucle si se alcanza el punto de detener
        if stop_training:
            break
    if stop_training:
        break

Training model for species: BANDERA, cluster: 7
Epoch 1/50
9264/9264 [==============================] - 61s 6ms/step - loss: 0.0012 - val_loss: 7.8446e-04
Epoch 2/50
9264/9264 [==============================] - 54s 6ms/step - loss: 0.0011 - val_loss: 7.0231e-04
Epoch 3/50
9264/9264 [==============================] - 54s 6ms/step - loss: 0.0011 - val_loss: 6.5556e-04
Epoch 4/50
9264/9264 [==============================] - 54s 6ms/step - loss: 0.0010 - val_loss: 6.5407e-04
Epoch 5/50
9264/9264 [==============================] - 54s 6ms/step - loss: 0.0010 - val_loss: 6.2270e-04
Epoch 6/50
9264/9264 [==============================] - 53s 6ms/step - loss: 0.0010 - val_loss: 6.3542e-04
Epoch 7/50
9264/9264 [==============================] - 53s 6ms/step - loss: 0.0010 - val_loss: 6.8005e-04
Epoch 8/50
9264/9264 [==============================] - 53s 6ms/step - loss: 0.0010 - val_loss: 6.7603e-04
Epoch 9/50
9264/9264 [==============================] - 53s 6ms/step - loss: 0.0010 - val_loss: 

/opt/conda/envs/cedo/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


33/33 [==============================] - 0s 4ms/step
MSE Mensual para BANDERA en clúster 7: 83180717.66826181, R2 Mensual: -1.9601911090489557


/var/tmp/ipykernel_29735/4225285831.py:60: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_totals = results_df.set_index('date').resample('M').sum()


Training model for species: BANDERA, cluster: 0
Epoch 1/50
41/41 [==============================] - 7s 31ms/step - loss: nan - val_loss: nan
Epoch 2/50
41/41 [==============================] - 0s 7ms/step - loss: nan - val_loss: nan
Epoch 3/50
41/41 [==============================] - 0s 7ms/step - loss: nan - val_loss: nan
Epoch 4/50
41/41 [==============================] - 0s 6ms/step - loss: nan - val_loss: nan
Epoch 5/50
41/41 [==============================] - 0s 6ms/step - loss: nan - val_loss: nan
Epoch 6/50
41/41 [==============================] - 0s 6ms/step - loss: nan - val_loss: nan
Epoch 7/50
41/41 [==============================] - 0s 6ms/step - loss: nan - val_loss: nan
Epoch 8/50
41/41 [==============================] - 0s 7ms/step - loss: nan - val_loss: nan
Epoch 9/50
41/41 [==============================] - 0s 7ms/step - loss: nan - val_loss: nan
Epoch 10/50
41/41 [==============================] - 0s 7ms/step - loss: nan - val_loss: nan
Epoch 10: early stopping
Model

...

/opt/conda/envs/cedo/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/50
5/5 [==============================] - 7s 257ms/step - loss: 0.2735 - val_loss: 0.0322
Epoch 2/50
5/5 [==============================] - 0s 16ms/step - loss: 0.1606 - val_loss: 0.0181
Epoch 3/50
5/5 [==============================] - 0s 14ms/step - loss: 0.1178 - val_loss: 0.0285
Epoch 4/50
5/5 [==============================] - 0s 14ms/step - loss: 0.0719 - val_loss: 0.0378
Epoch 5/50
5/5 [==============================] - 0s 14ms/step - loss: 0.0510 - val_loss: 0.0481
Epoch 6/50
5/5 [==============================] - 0s 13ms/step - loss: 0.0333 - val_loss: 0.0765
Epoch 7/50
5/5 [==============================] - 0s 13ms/step - loss: 0.0260 - val_loss: 0.1193
Epoch 8/50
5/5 [==============================] - 0s 14ms/step - loss: 0.0234 - val_loss: 0.1416
Epoch 9/50
5/5 [==============================] - 0s 14ms/step - loss: 0.0196 - val_loss: 0.1541
Epoch 10/50
5/5 [==============================] - 0s 14ms/step - loss: 0.0152 - val_loss: 0.1597
Epoch 11/50
5/5 [===========

/opt/conda/envs/cedo/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/50
5975/5975 [==============================] - 41s 6ms/step - loss: 0.0072 - val_loss: 0.0066
Epoch 2/50
5975/5975 [==============================] - 35s 6ms/step - loss: 0.0070 - val_loss: 0.0065
Epoch 3/50
4254/5975 [====================>.........] - ETA: 9s - loss: 0.0066

KeyboardInterrupt: 